In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
from owslib.csw import CatalogueServiceWeb
from owslib.fes import PropertyIsEqualTo, PropertyIsLike, BBox
import pandas as pd
from ipyaggrid import Grid

csw = CatalogueServiceWeb('http://explorer-portal.australiaeast.cloudapp.azure.com/geonetwork/srv/eng/csw')

permit_query = PropertyIsEqualTo('csw:AnyText', '%Permits%')
csw.getrecords2(constraints=[permit_query], maxrecords=200)

data = {}
attribs = ['identifier', 'title', 'abstract', 'type', 'subjects' ]
for key in csw.records:
    record = csw.records[key]
    for attrib in attribs:
        if attrib not in data.keys():
            data[attrib] = []
        data[attrib].append(getattr(record, attrib))

df_record = pd.DataFrame(data)

record_column_defs = [{'field': c, 'width': 300, 'editable': True} for c in df_record.columns]

record_grid_options = {
    'columnDefs' : record_column_defs,
    'enableFilter':'true',
    'enableSorting':'true'
}

g_record = Grid(
    grid_data=df_record,
    grid_options=record_grid_options,
    theme='ag-theme-balham',
    columns_fit='',
    quick_filter=True,
    export_csv=True,
    export_excel=True
)

g_record

Grid(compress_data=True, export_mode='disabled', height='350px', menu={'buttons': [{'name': 'Export to CSV', '…

In [7]:
from owslib.csw import CatalogueServiceWeb

csw.getrecordbyid(id=['aecdf0b2b48a07a8f95888d245781b9dc2a3faec', 'b194c58124d86f7cab8a1890e8e51971ec6443d9'])

record = csw.records['aecdf0b2b48a07a8f95888d245781b9dc2a3faec']
petro_permit_layer = record.uris[0]

petro_permit_layer

{'protocol': 'OGC:WFS-1.1.0-http-get-feature',
 'name': 'Permits_Petroleum:Petroleum_Active_Permits',
 'description': 'Petroleum_Active_Permits',
 'url': 'https://data.nzpam.govt.nz/hosting/services/Permits/Petroleum/MapServer/WFSServer?'}

In [8]:
record = csw.records['b194c58124d86f7cab8a1890e8e51971ec6443d9']
mineral_permit_layer = record.uris[0]

mineral_permit_layer

{'protocol': 'OGC:WFS-1.1.0-http-get-feature',
 'name': 'Permits_Minerals:Mineral_Active_Permits',
 'description': 'Mineral_Active_Permits',
 'url': 'https://data.nzpam.govt.nz/hosting/services/Permits/Minerals/MapServer/WFSServer?'}

In [9]:
import geopandas as gpd
from requests import Request
from owslib.wfs import WebFeatureService

params = dict(
    service='WFS', 
    version="2.0.0", 
    request='GetFeature', 
    typeName=petro_permit_layer['name'], 
    outputFormat='geojson', 
    srsname='urn:ogc:def:crs:EPSG::4326',
    bbox='173.56,-39.24,175.47,-36.44'
)
q = Request('GET', petro_permit_layer['url'], params=params).prepare().url

petroleum_df = gpd.read_file(q)

petroleum_df.head()

,OBJECTID,Permit_Number,PERMIT_NONEXCLUSIVE_YN,PERMIT_DURATION_YEARS,PERMIT_DURATION_MONTHS,PERMIT_AREA,PERMIT_AREA_UNITS,PERMIT_OFFSHORE_ONSHORE,Status,PERMIT_STATUS_DATE,...,Owner_s_,Share_Percentage_s_,Operator,Mineral_s_,B1_PER_ID1,B1_PER_ID2,B1_PER_ID3,SHAPE.STArea__,SHAPE.STLength__,geometry
0,15733,38150,N,60,0,44.360,SQKM,Onshore,Active,1/05/2001,...,Todd Energy Limited,100,Todd Energy Limited,"Gas, Petroleum, Condensate, LPG, Oil",00ATX,00000,00002,4.435287e+07,36318.692807,POLYGON ((174.2668677609472 -39.09796266603133...
1,10528,38154,N,32,0,155.800,SQKM,Offshore,Active,8/10/2004,...,"OMV New Zealand Limited, OMV NZ Production Lim...","25.99985, 48.0003, 25.99985",OMV NZ Production Limited,"Condensate, Gas, LPG, Petroleum",04ATX,00000,00002,1.558681e+08,61537.362819,"POLYGON ((174.249086413219 -38.81573649924178,..."
2,14533,38159,N,15,0,19.400,SQKM,Onshore,Active,5/04/2007,...,GREYMOUTH PETROLEUM CENTRAL LIMITED,100,GREYMOUTH PETROLEUM CENTRAL LIMITED,"Coal Seam Gas, Condensate, Gas, LPG, Oil, Petr...",05ATX,00000,00004,1.940396e+07,20571.183611,POLYGON ((174.2610364093767 -39.26213249960355...
3,6508,38161,N,30,0,45.889,SQKM,Onshore,Active - Change Pending,28/04/2006,...,"Greymouth Gas Kaimiro Limited, Greymouth Gas P...","30.1, 25, 25, 19.9",Greymouth Petroleum Turangi Limited,"Condensate, Gas, LPG, Oil, Petroleum",05ATX,00000,00006,4.590571e+07,37069.444099,POLYGON ((174.4002003627019 -38.97083333479793...
4,15333,57065,N,10,0,52.176,SQKM,Onshore,Active,1/04/2015,...,TAMARIND NZ ONSHORE LIMITED,100,TAMARIND NZ ONSHORE LIMITED,"Condensate, Gas, LPG, Oil, Petroleum",14ATX,00000,00140,5.219329e+07,44987.242742,POLYGON ((174.2208333318534 -39.14583333181125...


In [10]:
from ipywidgets import Text, HTML, Layout
import textwrap

from ipyleaflet import Map, GeoData, basemaps, LayersControl, WidgetControl

def click_handler (feature=None, **kwargs):
    if feature is not None:        
        html1.value = ''
        items = feature['properties']
        for item in items:
            html1.value += '<h6>{}: <b>{}</b></h6>'.format(item, textwrap.shorten(str(items[item]), width=40))
        
geo_data = GeoData(
    geo_dataframe=petroleum_df,
    style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    name='Petroleum Active Permits'
)
geo_data.on_click(click_handler)

m = Map(center=(-41.2865, 174.7762), zoom=6, layout=dict(height='800px'))
m.add_layer(geo_data)

html1 = HTML('')
html1.layout.margin = '0px 10px 10px 10px'
control1 = WidgetControl(widget=html1, position='bottomright')
m.add_control(control1)

m.add_control(LayersControl())

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [11]:
import geopandas as gpd
from requests import Request
from owslib.wfs import WebFeatureService

params = dict(
    service='WFS', 
    version="2.0.0", 
    request='GetFeature', 
    typeName='data.linz.govt.nz:layer-53564', 
    outputFormat='json',
    srsname='urn:ogc:def:crs:EPSG::4326',
    bbox='173.7165,-39.737,175.4963,-38.8528,EPSG:4326'
)
q = Request('GET', 'https://data.linz.govt.nz/services;key=e1cd2f7203d74a9ea5d179f269de890b/wfs', params=params).prepare().url

taranaki_rural_df = gpd.read_file(q)

taranaki_rural_df.head()

,id,napalis_id,start_date,name,recorded_area,overlays,type,legislation,section,reserve_purpose,ctrl_mg_vst,geometry
0,layer-53564.2991035,2991035,None,Mangapouri Cemetery - Local Purpose (Cemetery)...,20.4325,No,Reserve,Reserves Act 1977,s.23 - Local Purpose Reserve,Cemetery,New Plymouth District Council,(POLYGON ((174.1296076505625 -39.1350512713130...
1,layer-53564.3004179,3004179,None,Pukekura Park Local Purpose (Community Purpose...,1.1935,No,Reserve,Reserves Act 1977,s.23 - Local Purpose Reserve,Community Use,New Plymouth District Council,(POLYGON ((174.0882341009924 -39.0758864051503...
2,layer-53564.2802387,2802387,1978-01-01T00:00:00,New Plymouth Air Training Local Purpose Reserve,0.1238,No,Reserve,Reserves Act 1977,s.23 - Local Purpose Reserve,Air Training,Air Cadet League of New Zealand (Inc),(POLYGON ((174.0866527330251 -39.0819205546201...
3,layer-53564.2802388,2802388,1979-01-01T12:00:00,Stratford Air Training Local Purpose Reserve,0.1312,No,Reserve,Reserves Act 1977,s.23 - Local Purpose Reserve,Air Training,Air Cadet League of New Zealand Inc.,(POLYGON ((174.2884852667789 -39.3405695722368...
4,layer-53564.2802540,2802540,1894-01-01T00:00:00,Warea Cemetery Local Purpose Reserve,2.0234,No,Reserve,Reserves Act 1977,s.23 - Local Purpose Reserve,Cemetery,South Taranaki District Council,(POLYGON ((173.8088137000605 -39.2337633216579...


In [12]:
geo_data = GeoData(
    geo_dataframe=taranaki_rural_df,
    style={'color': 'blue', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    name='Conservation Layer'
)

m = Map(center=(-41.2865, 174.7762), zoom=6, layout=dict(height='800px'))
m.add_layer(geo_data)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [13]:
res_intersection = gpd.overlay(petroleum_df, taranaki_rural_df, how='intersection')

res_intersection.head()

,OBJECTID,Permit_Number,PERMIT_NONEXCLUSIVE_YN,PERMIT_DURATION_YEARS,PERMIT_DURATION_MONTHS,PERMIT_AREA,PERMIT_AREA_UNITS,PERMIT_OFFSHORE_ONSHORE,Status,PERMIT_STATUS_DATE,...,start_date,name,recorded_area,overlays,type,legislation,section,reserve_purpose,ctrl_mg_vst,geometry
0,15733,38150,N,60,0,44.360,SQKM,Onshore,Active,1/05/2001,...,1919-01-01T00:00:00,Everett Park Scenic Reserve,8.562630e+01,No,Reserve,Reserves Act 1977,s.19(1)(a) - Scenic Reserve,NULL,NULL,POLYGON ((174.2889151518796 -39.11074054484472...
1,15733,38150,N,60,0,44.360,SQKM,Onshore,Active,1/05/2001,...,1954-01-01T00:00:00,Awa te Take Scenic Reserve,2.266240e+01,No,Reserve,Reserves Act 1977,s.19(1)(a) - Scenic Reserve,NULL,NULL,POLYGON ((174.2982933669472 -39.06451508819668...
2,15733,38150,N,60,0,44.360,SQKM,Onshore,Active,1/05/2001,...,1905-01-01T00:00:00,Awa te Take Pa Historic Reserve,4.120200e+00,No,Reserve,Reserves Act 1977,s.18 - Historic Reserve,NULL,NULL,POLYGON ((174.2818199500427 -39.06217687133026...
3,10528,38154,N,32,0,155.800,SQKM,Offshore,Active,8/10/2004,...,2008-06-26T00:00:00,West Coast North Island Marine Mammal Sanctuary,1.193542e+06,No,Marine Area,Marine Mammals Protection Act 1978,s.22 - Marine Mammal Sanctuaries,NULL,NULL,"POLYGON ((174.249086413219 -38.81573649924178,..."
4,6508,38161,N,30,0,45.889,SQKM,Onshore,Active - Change Pending,28/04/2006,...,2008-06-26T00:00:00,West Coast North Island Marine Mammal Sanctuary,1.193542e+06,No,Marine Area,Marine Mammals Protection Act 1978,s.22 - Marine Mammal Sanctuaries,NULL,NULL,(POLYGON ((174.4002003627019 -38.9708333347979...


In [14]:
ax = res_intersection.plot(cmap='tab10')

In [15]:
inntersection_geo_data = GeoData(
    geo_dataframe=res_intersection,
    style={'color': 'blue', 'fillColor': '#3366cc', 'opacity':0.05, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
    name='Overlapping Area'
)

m = Map(center=(-41.2865, 174.7762), zoom=7, layout=dict(height='800px'))
m.add_layer(inntersection_geo_data)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [16]:
res_intersection.area

0     1.428761e-05
1     2.332105e-05
2     4.159023e-06
3     1.619583e-02
4     4.271991e-04
          ...     
67    4.547812e-06
68    1.334567e-06
69    5.039088e-07
70    5.241693e-07
71    2.398403e-06
Length: 72, dtype: float64